In [2]:
import tensorflow as tf
import os
import numpy as np
working_directory = "/home/jfeng/datasets/NMR/images"
data_file = "nmr_classification.csv"
image_directory = os.path.join(working_directory,"NMR_classification for SMN- Spectra_1")
import csv
csv_file = open(os.path.join(working_directory,data_file),"r")
csv_reader = csv.DictReader(csv_file)

In [3]:
class_names = []
image_names = []
classes = []
for rowDict in csv_reader:
    mol_name = rowDict['Molecule Name']
    img_file_name = rowDict['NMR_classification for SMN: Spectra File Name']
    kd = rowDict['NMR_classification for SMN: Kd (uM)']
    class_name = rowDict['NMR_classification for SMN: class'].strip()
    if class_name not in ['1','3']:
        continue
    class_name = int(class_name)
    if class_name not in class_names:
        class_names.append(class_name)
    classes.append(class_name)
    image_names.append(os.path.join(image_directory,img_file_name))
print(class_names)
for class_name in class_names:
    print(class_name)

[3, 1]
3
1


In [4]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
print(image_names)



['/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-15015-pre-classification-CDD-format-20191114.png', '/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-15012-pre-classification-CDD-format-20191114.png', '/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-14941-pre-classification-temp-CDD-format-20191021.png', '/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-14940-pre-classification-CDD-format-20191121.png', '/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-14940-pre-classification-temp-CDD-format-20191021.png', '/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-14939-pre-classification-CDD-format-20191121.png', '/home/jfeng/datasets/NMR/images/NMR_classification for SMN- Spectra_1/u1-smn2-titration-rgx-14939-pre-classification-temp-CD

In [5]:
print(len(image_names))
print(len(classes))
# import IPython.display as display
# from PIL import Image
# for image in image_names:
#     display.display(Image.open(image))

137
137


In [6]:
filenames = tf.constant(image_names)
labels = tf.constant(classes)
dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
X=128
Y=128
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_png(image_string)
    image_decoded = tf.image.resize(image_decoded,[X,Y])
    image_decoded = tf.reshape(image_decoded,[1,X,Y,3])
    image = tf.cast(image_decoded, tf.float32)
    label = tf.cast(label,tf.int32)
    label = tf.reshape(label,[1,1])
    return image, label
dataset = dataset.map(_parse_function)
#dataset = dataset.batch(2)
print("Size:",len(list(dataset)))

Size: 137


In [7]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(X, Y, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1280, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0

In [8]:
# length = len(list(dataset))
# train_size = int(0.8*length)
# train_dataset = dataset.shuffle(length).take(train_size)
# test_dataset = dataset.skip(train_size)
# # for e in train_dataset:
# #     print(e)
# # print(len(list(train_dataset)))
# # print(len(list(test_dataset)))
print("OK")
model.fit(dataset,epochs=10)
print("OK")

OK
Epoch 1/10
137/137 [==============================] - 5s 37ms/step - loss: -101422060990.8448 - accuracy: 0.2482

In [15]:
for d in dataset:
    print(d.numpy())

AttributeError: 'tuple' object has no attribute 'numpy'